<a href="https://colab.research.google.com/github/MichalMaczek/praca_inz/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import glob


# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))


from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

# 1.

**Get data**

In [ ]:
# os.chdir("/content/drive/MyDrive/Praca_Inż/Flows/Training")
# extension = 'binetflow'
# all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# combined_csv.to_csv( "training_flows.csv", index=False)

In [ ]:
# training_flows = '/content/drive/MyDrive/Inż/Training/training_flows.csv'
# df_start = pd.read_csv(training_flows)

JUST ONE SCENARIO

In [ ]:
training_flows = '/content/drive/MyDrive/Inż/Flows/Training/CTU-3.binetflow'
df_start = pd.read_csv(training_flows)

**CONSTANTS**


In [ ]:
protocols = ['tcp', 'udp', 'icmp']

# 2.

**Drop unnecessary features**

In [ ]:
df_start = df_start.drop(['StartTime', 'Dur', 'Dir', 'State', 'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes'], axis=1)
print(df_start)

        Proto          SrcAddr  ...  Dport                                   Label
0         tcp   74.125.108.243  ...  55707                         flow=Background
1         tcp    147.32.84.164  ...  15173              flow=From-Normal-V44-Grill
2         tcp  217.168.211.184  ...     22                         flow=Background
3         udp    173.9.132.155  ...  15046  flow=Background-Established-cmpgw-CVUT
4         tcp    88.103.40.117  ...  56306  flow=Background-Established-cmpgw-CVUT
...       ...              ...  ...    ...                                     ...
4710633   udp      147.32.85.2  ...     53  flow=To-Background-UDP-CVUT-DNS-Server
4710634   udp      147.32.85.2  ...     53  flow=To-Background-UDP-CVUT-DNS-Server
4710635   tcp      147.32.85.2  ...     80         flow=Background-TCP-Established
4710636   udp    89.176.63.143  ...  13363         flow=Background-UDP-Established
4710637   udp    81.33.153.210  ...  13363         flow=Background-UDP-Established

[47

**Separate internal network**

In [ ]:
# Seperate only 147.32.0.0/16 network
re_internal = '^147\.32\..*'
df_internal = df_start[df_start['SrcAddr'].str.match(re_internal)==True] 

In [ ]:
df_internal

,Proto,SrcAddr,Sport,DstAddr,Dport,Label
1,tcp,147.32.84.164,22,83.208.193.123,15173,flow=From-Normal-V44-Grill
5,tcp,147.32.84.242,22,84.242.66.38,49160,flow=Background
6,tcp,147.32.84.59,49973,89.233.174.25,443,flow=Background-Established-cmpgw-CVUT
7,tcp,147.32.87.245,52455,17.174.2.27,443,flow=Background
8,tcp,147.32.87.245,52456,17.174.2.27,443,flow=Background
...,...,...,...,...,...,...
4710629,tcp,147.32.84.59,58897,209.85.148.147,443,flow=Background-Established-cmpgw-CVUT
4710632,udp,147.32.84.229,13363,220.253.39.137,34262,flow=Background-UDP-Attempt
4710633,udp,147.32.85.2,55572,147.32.80.9,53,flow=To-Background-UDP-CVUT-DNS-Server
4710634,udp,147.32.85.2,61416,147.32.80.9,53,flow=To-Background-UDP-CVUT-DNS-Server


In [ ]:
hosts = df_internal['SrcAddr'].unique()
bots = df_internal[df_internal['Label'].str.contains('Botnet')]['SrcAddr'].unique()
print("No of all hosts in internal network: ", len(hosts))
print("No of all bots in internal network:", len(bots))

No of all hosts in internal network:  504
No of all bots in internal network: 1


# 3.

**Split flows depending on protocol**

In [ ]:
from IPython.display import clear_output
count = 1
flows = {}
for host in hosts:
  clear_output()
  print("Info: host no. ",count)
  flows[host] ={}
  flows_of_specified_host = df_internal.loc[df_internal['SrcAddr']==host]
  for protocol in protocols:
    flows[host][protocol] = flows_of_specified_host.loc[flows_of_specified_host['Proto']==protocol]
    if not flows[host][protocol].empty:
      flows[host][protocol] = flows[host][protocol].fillna(0)
  count +=1

Info: host no.  504


In [ ]:
## INFO
# display {all flows|flows specified by address|flows specified by address and protocol|all flows specified by protocol}

print(flows)
# print(flows['147.32.84.164'])
# print(flows['147.32.84.164']['icmp'])

# for addresses in flows:
#   print('\n')
#   print('Host: ' + str(addresses))
#   print(flows[addresses]['icmp'])


## display number of flows with specific host address
# for addresses in flows:
#   print("host: " + str(addresses) + ", " + str(len(flows[addresses]['icmp'])))  

{'147.32.3.93': {'tcp':   Proto      SrcAddr  ...  Dport                                   Label
7   tcp  147.32.3.93  ...  51790  flow=Background-Established-cmpgw-CVUT

[1 rows x 6 columns], 'udp': Empty DataFrame
Columns: [Proto, SrcAddr, Sport, DstAddr, Dport, Label]
Index: [], 'icmp': Empty DataFrame
Columns: [Proto, SrcAddr, Sport, DstAddr, Dport, Label]
Index: []}, '147.32.86.194': {'tcp':         Proto        SrcAddr  ... Dport                            Label
11        tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
69        tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3344      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3481      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3489      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
...       ...            ...  ...   ...                              ...
2638840   tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
2638919   tcp  1

# For later use

In [ ]:
# for each protocol, store the list of hosts with at least MIN_PKT_PROTO (150) packets
# e.g. for TCP, keep the list of hosts with at least MIN_PKT_PROTO TCP packets
hosts_proto = {}
for protocol in protocols:
  hosts_proto[protocol] = []
  for host in hosts:
    if(len(packets[host][protocol].index) >= 150):
      # print(host)
      # print(protocol)
      # print(len(packets[host][protocol].index))
      hosts_proto[protocol].append(host)
# hosts_proto

NameError: ignored

# 4.

**Convert hex values {ICMP Type, ICMP Code} to integers**

In [ ]:
# convert a hexadecimal number (e.g. for ICMP) to an integer file
def hexa2int(hexa):
    if str(hexa) == 'nan':
        return 0
    return int(str(hexa), 0)

# hexa2int('0')

In [ ]:
for addresses in flows:
  flows[addresses]['icmp']['Sport'] = flows[addresses]['icmp']['Sport'].apply(hexa2int)
  flows[addresses]['icmp']['Dport'] = flows[addresses]['icmp']['Dport'].apply(hexa2int)
  #flows[addresses]['icmp'].rename(columns={"Sport": "Type", "Dport": "Code"}, inplace=True)

In [ ]:
for addresses in flows:
  print('\n')
  print('Host: ' + str(addresses))
  print(flows[addresses]['icmp'])

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
55221    icmp  147.32.85.18  ...  49320  flow=Background-google-analytics3
56421    icmp  147.32.85.18  ...  49320                    flow=Background
1908963  icmp  147.32.85.18  ...  49320                    flow=Background
2206110  icmp  147.32.85.18  ...      0                    flow=Background
2432911  icmp  147.32.85.18  ...  49320                    flow=Background
3805403  icmp  147.32.85.18  ...  49320                    flow=Background
3851946  icmp  147.32.85.18  ...  49320                    flow=Background
3870681  icmp  147.32.85.18  ...  49320                    flow=Background
3983042  icmp  147.32.85.18  ...  49320                    flow=Background
4006933  icmp  147.32.85.18  ...  49320                    flow=Background
4158856  icmp  147.32.85.18  ...  49320                    flow=Background

[13 rows x 6 columns]


Host: 147.32.85.31
Empty DataFrame
Columns: [Proto, SrcAddr, Type, DstAddr, Code, Lab

In [ ]:
# check
print(flows)

{'147.32.3.93': {'tcp':   Proto      SrcAddr  ...  Dport                                   Label
7   tcp  147.32.3.93  ...  51790  flow=Background-Established-cmpgw-CVUT

[1 rows x 6 columns], 'udp': Empty DataFrame
Columns: [Proto, SrcAddr, Sport, DstAddr, Dport, Label]
Index: [], 'icmp': Empty DataFrame
Columns: [Proto, SrcAddr, Type, DstAddr, Code, Label]
Index: []}, '147.32.86.194': {'tcp':         Proto        SrcAddr  ... Dport                            Label
11        tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
69        tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3344      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3481      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
3489      tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
...       ...            ...  ...   ...                              ...
2638840   tcp  147.32.86.194  ...    80  flow=Background-TCP-Established
2638919   tcp  147

# For later use

**Convert IP addresses to integers**

In [ ]:
import struct
import socket
# convert the IP into an integer, e.g. IP 255.255.255.255 = 255*(256^3) + 255*(256^2) + 255*256 + 255 = 4294967295
def ip2int(addr):
    try:
        _str = socket.inet_aton(addr)
    except socket.error:
        raise ValueError
    return struct.unpack('!I', _str)[0]


In [ ]:
for addresses in flows:
  for protocol in protocols:
    flows[addresses][protocol]['SrcAddr'] = flows[addresses][protocol]['SrcAddr'].apply(ip2int)
    flows[addresses][protocol]['DstAddr'] = flows[addresses][protocol]['DstAddr'].apply(ip2int)

In [ ]:
# check
print(flows)

{'147.32.84.164': {'tcp':         Proto     SrcAddr  Sport     DstAddr  Dport                       Label
1         tcp  2468369572     22  1406189947  15173  flow=From-Normal-V44-Grill
1083362   tcp  2468369572  48018  1539135519     80  flow=From-Normal-V44-Grill
1083368   tcp  2468369572  53344  1539136678     80  flow=From-Normal-V44-Grill
1083372   tcp  2468369572  57230  1539135521     80  flow=From-Normal-V44-Grill
1083375   tcp  2468369572  57231  1539135521     80  flow=From-Normal-V44-Grill
...       ...         ...    ...         ...    ...                         ...
4644386   tcp  2468369572  50485  1249765581     80  flow=From-Normal-V44-Grill
4644396   tcp  2468369572  54161  3512046950     80  flow=From-Normal-V44-Grill
4660607   tcp  2468369572  54826  1249765569    443  flow=From-Normal-V44-Grill
4681279   tcp  2468369572  52528  1249765580    443  flow=From-Normal-V44-Grill
4702070   tcp  2468369572  45317  1249765571    443  flow=From-Normal-V44-Grill

[1187 rows x 

# 5.

**Host signatures**

80


In [ ]:
attributes = ['Sport', 'DstAddr', 'Dport']

In [ ]:
no_of_features = 9
no_of_bins = 32
signature = [0 for i in range(no_of_features * no_of_bins)]

In [ ]:
max_port = 65535
signatures = {}
for host in hosts:
  signatures[host] = {}
  signatures[host] = signature
  for protocol in protocols:
    for attribute in attributes:
      if 'port' in flows[host][protocol][attribute].name:
        for i in range(len(flows[host][protocol][attribute])):
          print(i)
          temp = int(flows[host][protocol][attribute].values[i])
          temp = int(temp/((max_port + 1)/no_of_bins))
          signatures[host][temp] += 1
        # for i in range(0, max_port + 2, int((max_port + 1)/no_of_bins)):
        #   print(i)
          #print(flows[host][protocol][attribute])
          # if flows[host][protocol][attribute] <
          # signature[host][proto + '_' + attribute].append(i)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
46684
46685
46686
46687
46688
46689
46690
46691
46692
46693
46694
46695
46696
46697
46698
46699
46700
46701
46702
46703
46704
46705
46706
46707
46708
46709
46710
46711
46712
46713
46714
46715
46716
46717
46718
46719
46720
46721
46722
46723
46724
46725
46726
46727
46728
46729
46730
46731
46732
46733
46734
46735
46736
46737
46738
46739
46740
46741
46742
46743
46744
46745
46746
46747
46748
46749
46750
46751
46752
46753
46754
46755
46756
46757
46758
46759
46760
46761
46762
46763
46764
46765
46766
46767
46768
46769
46770
46771
46772
46773
46774
46775
46776
46777
46778
46779
46780
46781
46782
46783
46784
46785
46786
46787
46788
46789
46790
46791
46792
46793
46794
46795
46796
46797
46798
46799
46800
46801
46802
46803
46804
46805
46806
46807
46808
46809
46810
46811
46812
46813
46814
46815
46816
46817
46818
46819
46820
46821
46822
46823
46824
46825
46826
46827
46828
46829
46830
46831
46832
46833
46834
46835
46836
46837
46838
46839

KeyboardInterrupt: ignored

In [ ]:
print(signatures)

{'147.32.84.164': [340659, 116752, 53063, 1246, 1989, 1013, 920, 8657, 1571, 1385, 634, 2911, 2883, 2527, 1182, 559, 7641, 7109, 7547, 7000, 8090, 8550, 7207, 7179, 31669, 22344, 16995, 16075, 17102, 16987, 12485, 9686, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '147.32.84.